### Trabalho Prático 2 
__Nome:__ Gabriela Tavares Barreto 

__Matrícula:__ 2018074657

In [159]:
import numpy as np
import networkx as nx
import heapq as hq
import matplotlib.pyplot as plt
import sys
import math

INF = float(sys.maxsize)

In [161]:
def ler_arquivo(arq):
    arquivo = open(arq, 'r')
    vertices = []
    flag = False
    for linha in arquivo:
        if not flag:
            if linha.split()[0] == 'DIMENSION':
                d = int(linha.split()[2])
            if linha == 'NODE_COORD_SECTION\n':
                flag = True
                continue
        if linha == 'EOF\n':
            break

        if flag:
            vertice = [0, (0,0)]
            lista_num = linha.split()
            vertice[0] = int(lista_num[0])

            if vertice[0] == 20:
               break

            vertice[1] = (int(lista_num[1]), int(lista_num[2]))
            vertices.append(vertice)
    
    arquivo.close()

    return d, vertices


In [162]:

d, dados = ler_arquivo('./instancias/eil51.tsp')


In [163]:
def distancia_euclidiana(a, b):
    return np.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2)

def calcula_arestas(d, dados):
    arestas = []
    for i in range(0, d-1):
        for j in range(i+1, d):
            aresta = [0, 0, 0]
            aresta[0] = dados[i][0]  # vertice 1
            aresta[1] = dados[j][0]  # vertice 2
            aresta[2] = distancia_euclidiana(dados[i][1], dados[j][1])  # peso
            arestas.append(aresta)
    return arestas


In [164]:
#d = 9
arestas = calcula_arestas(d, dados)

In [165]:
grafo = nx.Graph()
grafo.add_weighted_edges_from(arestas)


In [167]:
def bound(G, c):
    bound = 0
    n = G.number_of_nodes()

    # Adicionar custos das arestas no caminho atual
    for i in range(len(c) - 1):
        if c[i+1] == 0:
            break
        
        bound += G[c[i]][c[i+1]]['weight']

    # Estimativa para nós não visitados
    nao_visitado = set(G.nodes()) - set(c)

    for node in nao_visitado:
        
        if nao_visitado - {node}:
            min_edge = min([G[node][neigh]['weight'] for neigh in nao_visitado - {node}])
        else:
        # Lidar com o conjunto vazio, por exemplo, definindo min_edge como 0 ou INF
            min_edge = 0

        # Menor aresta conectando o nó a qualquer outro nó não visitado
        #min_edge = min([G[node][neigh]['weight'] for neigh in nao_visitado - {node}])

        bound += min_edge

    bound = math.ceil(bound/2)
    
    return bound

class Node:
    def __init__(self, bound, caminho, nivel):
        self.bound = bound
        self.caminho = caminho
        self.nivel = nivel

    def __lt__(self, outro):
        return self.bound < outro.bound
    
    
def no_caminho(c, v):
    for i in c:
        if i == v:
            return True
    return False
        
def custo_caminho(c, G):
    custo = 0
    print(c)
    for i in range(-1,len(c)-2):
        custo += G[c[i]][c[i+1]]['weight']
    
    return custo

def branch_and_bound(G, N):
    c = np.zeros(N, dtype=int) #caminho vazio
    c[0] = 1
    b = bound(G, c) # limite inferior da raiz
    raiz = Node(b, c, 1) # bound, caminho, nível da arvore
    
    h = []
    hq.heappush(h, raiz)
    melhor = INF
    sol = []

    while(len(h) != 0):
        node = hq.heappop(h)
        if node.nivel > N+1:
            #print("oi")
            custo_sol = custo_caminho(node.caminho, G)
            if melhor > custo_sol:
                melhor = custo_sol
                sol = node.caminho
        elif node.bound < melhor:
            if node.nivel < N:
                for j in range(1, N+1):
                    c_novo = node.caminho.copy()
                    c_novo[node.nivel] = j
                    if not no_caminho(node.caminho, j):
                        novo_bound = bound(G, c_novo)
                        #print(c_novo)
                        #print(novo_bound)
                        if novo_bound < melhor:
                            aux_node = Node(novo_bound, c_novo, node.nivel+1)
                            hq.heappush(h, aux_node)
            else: # fechando o circuito
                if custo_caminho(node.caminho, G) < melhor: 
                    aux_node = Node(node.bound, node.caminho, node.nivel+1)
                    #print(node.nivel+1)
                    hq.heappush(h, aux_node)
    return melhor, sol

In [168]:
branch_and_bound(grafo, d)

KeyboardInterrupt: 

In [153]:
tsp = nx.approximation.traveling_salesman_problem
path = tsp(grafo, cycle=True)

In [158]:
custo_caminho(path[:8], grafo)

[1, 2, 3, 9, 5, 4, 6, 7]


135.28099081151072